In [ ]:
import tensorflow as tf
import numpy as np
from music21 import *
import os
import json
import numpy as np
import pickle
from random import shuffle
import zipfile
num_notes = 128
length = 19200
noisesize = 19200
learning_rate = 0.001

In [ ]:
def discriminator(X, num_filters, reuse = None):
    with tf.variable_scope('dis',reuse=reuse):
        layer1 = tf.layers.conv2d(inputs = X, filters = num_filters, kernel_size = [48,16], padding = "same", activation = tf.nn.relu, strides = [24,1])
        layer2 = tf.layers.conv2d(inputs = layer1, filters = num_filters*2, kernel_size = [16,8], padding = "same", activation = tf.nn.relu, strides = [2,2])
        layer3 = tf.layers.conv2d(inputs = layer2, filters = num_filters*4, kernel_size = [8,4], padding = "same", activation = tf.nn.relu, strides = [1,2])
        flat = tf.reshape(layer3, [-1, int(num_notes/192 * length * num_filters*4)])
        dense = tf.layers.dense(inputs=flat, units=64, activation=tf.nn.relu)
        dropout = tf.layers.dropout(inputs=dense, rate=0.4)
        logits = tf.layers.dense(inputs=dropout, units=1)
        output = tf.sigmoid(logits)
        return output, logits
    
        

In [ ]:
def generator(z, num_neurons, reuse = None):
    with tf.variable_scope('gen',reuse=reuse):
        cell = tf.contrib.rnn.OutputProjectionWrapper(
        tf.contrib.rnn.BasicLSTMCell(num_units=num_neurons, activation=tf.nn.relu6),
        output_size=num_notes)
        outputs, states = tf.nn.dynamic_rnn(cell, z[:,:,:], dtype=tf.float32)
        scalar = (1/6)
        divided = tf.math.scalar_mul(scalar, outputs)
        #lastoutput = outputs[:,-1,:]
        #bias_tensor = tf.fill(tf.shape(lastoutput), .5-a)
        #lastoutput = tf.add(lastoutput, bias_tensor)
        #lastoutput = tf.round(lastoutput)
        #reshapedlastout = tf.reshape(lastoutput,[-1, 1 , num_notes])
        #reshapedlastout = reshapedlastout/6
        #z = tf.concat([z,reshapedlastout],1)
        #for i in range(length-1):
        #    print(i)
        #    outputs, states = tf.nn.dynamic_rnn(cell, z[:,:,-noisesize:], dtype=tf.float32)
        #    lastoutput = outputs[:,-1,:]
        #    reshapedlastout = tf.reshape(lastoutput,[-1,1 , num_notes])
        #    reshapedlastout = reshapedlastout/6
        #    z = tf.concat([z,reshapedlastout],1)
        #z = z[:,-length:,:]
        return tf.reshape(divided, [-1,19200,128,1])

In [ ]:
def reconstruct(array, filename):
    ray = array
    #print(ray.shape)
    #ray = ray.reshape(19200,128)
    ray = ray.reshape(128,19200)
    print(ray[:,5])
    stream1 = stream.Stream()
    for x in range(ray.shape[1]):
        for y in range(ray.shape[0]):
            if (ray[y,x] == 1) and (ray[y,x-1] == 0):
                w = 0
                while ray[y, x+w] == 1:
                    w += 1
                    if x+w >= 19200:
                        break
                w += 2
                note1 = note.Note(y) #check this
                note1.quarterLength = w/24
                stream1.append(note1)
                note1.offset = x/24 #is this the right way?
    stream1.write("midi", "./sampleMusic/" + filename + ".mid")

In [ ]:
def next_batch(lis):
    counter = 0
    string = ""
    for item in lis:
        string = str(item)
        zipfilePath = ("./numpys/" + string + ".pickle.zip")
        zip = zipfile.ZipFile(zipfilePath)
        zip.extractall("./numpys")
        pickle_file = open("./numpys/" + string + ".pickle","rb")
        lis = pickle.load(pickle_file)
        pickle_file.close()
        ray = np.array(lis)
        if counter == 0:
            #print("made final")
            final = np.reshape(ray, [1,length,num_notes,1])
        else:
            #print("catted")
            final = np.concatenate((final, np.reshape(ray, [1,length,num_notes,1])), 0)
            #print(final.shape)
        os.remove("./numpys/" + string + ".pickle")
        counter = counter + 1
    return (final)

In [ ]:
#with tf.device('/gpu:0'):
real_music = tf.placeholder(tf.float32, shape = [None, length, num_notes,1])
z = tf.placeholder(tf.float32,shape=[None,noisesize, num_notes])

In [ ]:
#with tf.device('/gpu:0'):
g = generator(z,128)

In [ ]:
#with tf.device('/gpu:0'):
D_output_real , D_logits_real = discriminator(real_music, 16)
D_output_fake, D_logits_fake = discriminator(g,16,reuse=True)

In [ ]:
def loss_func(logits_in,labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in,labels=labels_in))

In [ ]:
#with tf.device('/gpu:0'):
D_real_loss = loss_func(D_logits_real,tf.ones_like(D_logits_real)* (0.9))
D_fake_loss = loss_func(D_logits_fake,tf.zeros_like(D_logits_real))
D_loss = D_real_loss + D_fake_loss
G_loss = loss_func(D_logits_fake,tf.ones_like(D_logits_fake))

In [ ]:
tvars = tf.trainable_variables()

d_vars = [var for var in tvars if 'dis' in var.name]
g_vars = [var for var in tvars if 'gen' in var.name]

print([v.name for v in d_vars])
print([v.name for v in g_vars])

In [ ]:
#with tf.device('/gpu:0'):
D_trainer = tf.train.AdamOptimizer(learning_rate).minimize(D_loss, var_list=d_vars)
G_trainer = tf.train.AdamOptimizer(learning_rate).minimize(G_loss, var_list=g_vars)

In [ ]:
batch_size = 32
num_songs = 20425
epochs = 100
init = tf.global_variables_initializer()
saver = tf.train.Saver(var_list=g_vars)

In [ ]:
tf.summary.scalar("D loss", D_loss)
tf.summary.scalar("G loss", G_loss)
merged_summary_op = tf.summary.merge_all()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    print("inited")
    writer = tf.summary.FileWriter("./editedoutput",sess.graph)
    for e in range(epochs):
        num_batches = num_songs // batch_size
        lis = list(range(num_songs))
        shuffle(lis)
        for i in range(num_batches):
            #print(i)
            batch = next_batch(lis[i*batch_size:i*batch_size + batch_size])
            
            
            batch_z = np.random.uniform(0, 1, size=[batch_size, noisesize, num_notes]) #not sure if 0 or -1
            
            _ = sess.run(D_trainer, feed_dict={real_music: batch, z: batch_z})
            _ = sess.run(G_trainer, feed_dict={z: batch_z})
            summaries = sess.run(merged_summary_op, feed_dict={real_music: batch, z: batch_z})
            writer.add_summary(summaries, e * num_batches + i)
            if i%1 == 0:
                print("32 done!")
                #losd = sess.run(D_loss,feed_dict={real_music: batch_data, z: batch_z})
                #losg = sess.run(G_loss,feed_dict={real_music: batch_data, z: batch_z})
                #print("Generator loss: " + str(losg))
                #print("Discriminatior loss: " + str(losd))
            if i%10 == 0:
                sample_z = np.random.uniform(0, 1, size=[1, noisesize, num_notes]) 
                gen_sample = sess.run(generator(z,128,reuse = True),feed_dict={z: sample_z})
                reconstruct(gen_sample, str(e) + "," +str(i))
                saver.save(sess, "model.ckpt")
        print("Currently on Epoch {} of {} total...".format(e+1, epochs))
        sample_z = np.random.uniform(0, 1, size=[1, noisesize, num_notes]) 
        gen_sample = sess.run(generator(z,128, reuse = True),feed_dict={z: sample_z})
        reconstruct(gen_sample, e)
        saver.save(sess, './checkpoints/model.ckpt')
